In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from torch.autograd import Variable

print(torch.__version__)
print(torch.cuda.is_available())
# device = torch.device('cpu')

In [ ]:
class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
#         assert indices.requires_grad == False
        a = torch.sparse_coo_tensor(indices, values, shape)
        ctx.save_for_backward(a, b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.matmul(b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = a.t().matmul(grad_output)
        return None, grad_values, None, grad_b

class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)


In [ ]:
class SpGraphAttentionLayer(nn.Module):
    """
    Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(SpGraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_normal_(self.W.data, gain=1.414)
                
        self.a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

        self.dropout = nn.Dropout(dropout)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.special_spmm = SpecialSpmm()

    def forward(self, input, adj, edge=None):
        dv = 'cuda' if input.is_cuda else 'cpu'

        N = input.size()[0]
        if edge is None:
            edge = adj.nonzero()
        edge = edge.t()

        h = torch.mm(input, self.W)
        # h: N x out
#         assert not torch.isnan(h).any()

        # Self-attention on the nodes - Shared attention mechanism
        edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1).t()
        # edge: 2*D x E

        edge_e = torch.exp(-self.leakyrelu(self.a.mm(edge_h).squeeze()))
#         assert not torch.isnan(edge_e).any()
        # edge_e: E

        e_rowsum = self.special_spmm(edge, edge_e, torch.Size([N, N]), torch.ones(size=(N,1), device=dv))
        # e_rowsum: N x 1

        edge_e = self.dropout(edge_e)
        # edge_e: E

        h_prime = self.special_spmm(edge, edge_e, torch.Size([N, N]), h)
#         assert not torch.isnan(h_prime).any()
        # h_prime: N x out
        
        h_prime = h_prime.div(e_rowsum)
        # h_prime: N x out
#         assert not torch.isnan(h_prime).any()

        if self.concat:
            # if this layer is not last layer,
            return F.elu(h_prime)
        else:
            # if this layer is last layer,
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [4]:
class SpGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Sparse version of GAT."""
        super(SpGAT, self).__init__()
        self.dropout = dropout

        self.attentions = [SpGraphAttentionLayer(nfeat, 
                                                 nhid, 
                                                 dropout=dropout, 
                                                 alpha=alpha, 
                                                 concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = SpGraphAttentionLayer(nhid * nheads, 
                                             nclass, 
                                             dropout=dropout, 
                                             alpha=alpha, 
                                             concat=False)

    def forward(self, x, adj, edges=None):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj, edges) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj, edges))
        return F.log_softmax(x, dim=1)

In [5]:
sparse_adj, sparse_adj_train,sparse_adj_train_all,features, train_feature,train_feature_all,labels, train_labels, id_train, id_valid, id_test, num_labels = Origin_load_ether_data()
sparse_adj.setdiag(1.0)
sparse_adj_train.setdiag(1.0)
sparse_adj_train_all.setdiag(1.0)

coo = sparse_adj.tocoo()
values = coo.data
indices = np.vstack((coo.row, coo.col)).astype(np.int32)
_i = torch.LongTensor(indices)
_v = torch.FloatTensor(values)
shape = coo.shape
adj = torch.sparse.FloatTensor(_i, _v, torch.Size(shape))
# dataset splits
idx_train = np.array(np.zeros(features.shape[0]), dtype=np.bool)
idx_val = np.array(np.zeros(features.shape[0]), dtype=np.bool)
idx_test = np.array(np.zeros(features.shape[0]), dtype=np.bool)
id_train = np.array(id_train)
id_valid = np.array(id_valid)
id_test = np.array(id_test)
idx_train[id_train] = True
idx_val[id_valid] = True
idx_test[id_test] = True
idx_train = torch.tensor(idx_train)
idx_val = torch.tensor(idx_val)
idx_test = torch.tensor(idx_test)

edges = load_ether_edges()
edges = torch.tensor(edges, dtype=torch.int64)

features = (1402220, 4) type = <class 'numpy.ndarray'>
train_index 572
val_index 82
test_index 162
adj torch.Size([1402220, 1402220])
features torch.Size([1402220, 4])


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


edges = [[      0       1]
 [      0     803]
 [      0     893]
 ...
 [1396842  513010]
 [1396843   12104]
 [1396844  213600]]
type = <class 'numpy.ndarray'>
shape = (2812994, 2)


In [6]:
hidden, dropout, nb_heads, alpha = 8, 0.6, 8, 0.2
cuda = True
lr, weight_decay = 0.005, 5e-4
model = SpGAT(nfeat=features.shape[1], 
            nhid=hidden, 
            nclass=num_labels, 
            dropout=dropout, 
            nheads=nb_heads, 
            alpha=alpha)

optimizer = optim.Adam(model.parameters(), 
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model = model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    edges = edges.cuda()

# if True:
#     model = model.to(device)
#     features = features.to(device)
#     adj = adj.to(device)
#     labels = labels.to(device)
#     idx_train = idx_train.to(device)
#     idx_val = idx_val.to(device)
#     idx_test = idx_test.to(device)

features, adj, labels = Variable(features), Variable(adj), Variable(labels)


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj, edges)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    model.eval()
    output = model(features, adj, edges)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()


def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data[0]),
          "accuracy= {:.4f}".format(acc_test.data[0]))

In [7]:
# Train model
epochs, patience = 10000, 100
t_total = time.time()
loss_values = []
bad_counter = 0
best = epochs + 1
best_epoch = 0
for epoch in range(epochs):
    loss_values.append(train(epoch))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == patience:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
compute_test()

RuntimeError: CUDA out of memory. Tried to allocate 7324.74 GiB (GPU 0; 10.76 GiB total capacity; 3.58 GiB already allocated; 6.00 GiB free; 3.77 GiB reserved in total by PyTorch)